In [51]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import Imputer
from sklearn.metrics import confusion_matrix
from sklearn.grid_search import GridSearchCV
pd.options.display.max_columns = 100

In [47]:
df=pd.read_csv("parsed_df/parsed_df.csv")

In [61]:
dftouse=df.copy()
dftouse=dftouse.fillna(0)
dftouse.head()

,Unnamed: 0,EVENTMSGACTIONTYPE,EVENTMSGTYPE,EVENTNUM,GAME_ID,PERIOD_x,PERSON1TYPE,PERSON2TYPE,PERSON3TYPE,PLAYER2_ID,PLAYER3_ID,SCOREMARGIN,WCTIMESTRING,HOME_TEAM_ID,AWAY_TEAM_ID,HOME_PLAYER1_ID,HOME_PLAYER2_ID,HOME_PLAYER3_ID,HOME_PLAYER4_ID,HOME_PLAYER5_ID,AWAY_PLAYER1_ID,AWAY_PLAYER2_ID,AWAY_PLAYER3_ID,AWAY_PLAYER4_ID,AWAY_PLAYER5_ID,ACTION_BEFORE,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,DRIBBLES,FINAL_MARGIN,LOCATION,PTS,PTS_TYPE,SHOT_CLOCK,SHOT_DIST,SHOT_NUMBER,SHOT_RESULT,TOUCH_TIME,W,GAME_EVENT_ID,PLAYER_ID,TEAM_ID,ACTION_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_MADE_FLAG,off_height,def_height,SEC_LEFT_Q,SEC_LEFT_G
0,0,1,2,2,21300001,1,4,0,0,0,0,0,7:12 PM,1610612754,1610612753,2561,201579,201588,202331,202362,2749,101131,201167,202696,203090,10,101131,1.6,2,10,1,0,2,6.7,7.4,1,missed,7.8,True,2,2561,1610612754,Jump Shot,In The Paint (Non-RA),Center(C),Less Than 8 ft.,5,-38,45,0,81,79,699,2859
1,1,79,1,5,21300001,1,4,0,0,0,0,2,7:13 PM,1610612754,1610612753,2561,201579,201588,202331,202362,2749,101131,201167,202696,203090,6,203090,4.1,1,10,1,2,2,16.8,19.3,1,made,3.2,True,5,202331,1610612754,Pullup Jump shot,Mid-Range,Right Side Center(RC),16-24 ft.,19,105,164,1,81,81,690,2850
2,2,1,2,6,21300001,1,5,0,0,0,0,-2,7:13 PM,1610612754,1610612753,2561,201579,201588,202331,202362,2749,101131,201167,202696,203090,1,202331,4.4,0,-10,0,0,3,13.0,24.5,1,missed,0.7,False,6,201167,1610612753,Jump Shot,Above the Break 3,Center(C),24+ ft.,27,51,266,0,77,81,675,2835
3,3,5,2,8,21300001,1,4,0,5,0,101131,2,7:13 PM,1610612754,1610612753,2561,201579,201588,202331,202362,2749,101131,201167,202696,203090,4,101131,0.4,0,10,1,0,2,13.8,1.4,2,missed,1.8,True,8,2561,1610612754,Layup Shot,Restricted Area,Center(C),Less Than 8 ft.,2,28,-5,0,81,79,663,2823
4,4,1,1,11,21300001,1,4,4,0,2561,0,5,7:14 PM,1610612754,1610612753,2561,201579,201588,202331,202362,2749,101131,201167,202696,203090,5,2749,5.7,0,10,1,3,3,15.6,25.1,1,made,0.7,True,11,202362,1610612754,Jump Shot,Above the Break 3,Center(C),24+ ft.,26,15,260,1,77,72,651,2811


In [62]:
#create train and test set
from sklearn.cross_validation import train_test_split
itrain, itest = train_test_split(xrange(dftouse.shape[0]), train_size=0.7)
mask=np.ones(dftouse.shape[0], dtype='int')
mask[itrain]=1
mask[itest]=0
mask = (mask==1)

In [63]:
dftouse.isnull().any()

Unnamed: 0                    False
EVENTMSGACTIONTYPE            False
EVENTMSGTYPE                  False
EVENTNUM                      False
GAME_ID                       False
PERIOD_x                      False
PERSON1TYPE                   False
PERSON2TYPE                   False
PERSON3TYPE                   False
PLAYER2_ID                    False
PLAYER3_ID                    False
SCOREMARGIN                   False
WCTIMESTRING                  False
HOME_TEAM_ID                  False
AWAY_TEAM_ID                  False
HOME_PLAYER1_ID               False
HOME_PLAYER2_ID               False
HOME_PLAYER3_ID               False
HOME_PLAYER4_ID               False
HOME_PLAYER5_ID               False
AWAY_PLAYER1_ID               False
AWAY_PLAYER2_ID               False
AWAY_PLAYER3_ID               False
AWAY_PLAYER4_ID               False
AWAY_PLAYER5_ID               False
ACTION_BEFORE                 False
CLOSEST_DEFENDER_PLAYER_ID    False
CLOSE_DEF_DIST              

In [64]:
shooter = [u'ACTION_BEFORE', u'CLOSEST_DEFENDER_PLAYER_ID', u'CLOSE_DEF_DIST', u'DRIBBLES', u'GAME_ID', u'LOCATION', u'LOC_X', u'LOC_Y', u'PERIOD_x', u'PLAYER_ID', u'PTS_TYPE', u'SCOREMARGIN', u'SEC_LEFT_G', u'SEC_LEFT_Q',  u'SHOT_CLOCK', u'SHOT_DISTANCE', u'SHOT_NUMBER', u'TEAM_ID', u'TOUCH_TIME']
defender = [u'ACTION_BEFORE', u'CLOSEST_DEFENDER_PLAYER_ID', u'CLOSE_DEF_DIST', u'GAME_ID', u'LOCATION', u'LOC_X', u'LOC_Y', u'PERIOD_x', u'PLAYER_ID', u'PTS_TYPE', u'SCOREMARGIN', u'SEC_LEFT_G', u'SEC_LEFT_Q',  u'SHOT_CLOCK', u'SHOT_DISTANCE', u'TEAM_ID']

In [65]:
from sklearn.svm import LinearSVC
clfsvm=LinearSVC(loss="hinge")
Cs=[0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
Xmatrix=dftouse[shooter].values
Yresp=dftouse['SHOT_MADE_FLAG'].values

In [66]:
def cv_optimize(clf, parameters, X, y, n_folds=5, score_func=None):
    if score_func:
        gs = GridSearchCV(clf, param_grid=parameters, cv=n_folds, scoring=score_func)
    else:
        gs = GridSearchCV(clf, param_grid=parameters, cv=n_folds)
    gs.fit(X, y)
    print "BEST PARAMS", gs.best_params_
    best = gs.best_estimator_
    return best

def do_classify(clf, parameters, indf, featurenames, targetname, target1val, mask=None, reuse_split=None, score_func=None, n_folds=5):
    subdf=indf[featurenames]
    X=subdf.values
    y=(indf[targetname].values==target1val)*1
    if mask !=None:
        print "using mask"
        Xtrain, Xtest, ytrain, ytest = X[mask], X[~mask], y[mask], y[~mask]
    if reuse_split !=None:
        print "using reuse split"
        Xtrain, Xtest, ytrain, ytest = reuse_split['Xtrain'], reuse_split['Xtest'], reuse_split['ytrain'], reuse_split['ytest']
    if parameters:
        clf = cv_optimize(clf, parameters, Xtrain, ytrain, n_folds=n_folds, score_func=score_func)
    clf=clf.fit(Xtrain, ytrain)
    training_accuracy = clf.score(Xtrain, ytrain)
    test_accuracy = clf.score(Xtest, ytest)
    print "############# based on standard predict ################"
    print "Accuracy on training data: %0.2f" % (training_accuracy)
    print "Accuracy on test data:     %0.2f" % (test_accuracy)
    print confusion_matrix(ytest, clf.predict(Xtest))
    print "########################################################"
    return clf, Xtrain, ytrain, Xtest, ytest

In [67]:
%%time
clfsvm, Xtrain, ytrain, Xtest, ytest = do_classify(LinearSVC(loss="hinge"), {"C": [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]}, dftouse,shooter, u'SHOT_MADE_FLAG',1, mask=mask)

using mask


/Users/raymondcen/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:15: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


KeyboardInterrupt: 